In [ ]:
import google.generativeai as genai
import PyPDF2
import os
import tkinter as tk
from tkinter import filedialog, ttk, scrolledtext, messagebox

genai.configure(api_key="Your Gemini API Key")

class SummarizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Gemini Text Summarizer")
        self.root.geometry("800x600")
        self.root.config(padx=20, pady=20)
        
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill=tk.BOTH, expand=True, pady=10)
        
        self.text_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.text_tab, text="Text Input")
        
        self.pdf_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.pdf_tab, text="PDF Upload")
        
        self.setup_text_tab()
        
        self.setup_pdf_tab()
        
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        status_bar = ttk.Label(root, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        self.pdf_text = ""
    
    def setup_text_tab(self):
        # Title
        title_label = ttk.Label(self.text_tab, text="Gemini Text Summarizer", font=("Helvetica", 16, "bold"))
        title_label.pack(pady=10)
        
        # Instructions
        instructions = ttk.Label(self.text_tab, text="Enter the text you want to summarize:")
        instructions.pack(anchor=tk.W, padx=10)
        
        # Text input area
        self.text_input = scrolledtext.ScrolledText(self.text_tab, height=10, wrap=tk.WORD)
        self.text_input.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Summary button
        self.text_summarize_button = ttk.Button(self.text_tab, text="Summarize", 
                                               command=lambda: self.summarize("text"))
        self.text_summarize_button.pack(pady=10)
        
        # Output area
        output_frame = ttk.LabelFrame(self.text_tab, text="Summary")
        output_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        self.text_output = scrolledtext.ScrolledText(output_frame, height=8, wrap=tk.WORD)
        self.text_output.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    def setup_pdf_tab(self):
        # Title
        title_label = ttk.Label(self.pdf_tab, text="Gemini PDF Summarizer", font=("Helvetica", 16, "bold"))
        title_label.pack(pady=10)
        
        # PDF selection area
        pdf_frame = ttk.Frame(self.pdf_tab)
        pdf_frame.pack(fill=tk.X, padx=10, pady=20)
        
        self.pdf_label = ttk.Label(pdf_frame, text="No PDF selected")
        self.pdf_label.pack(side=tk.LEFT, padx=5)
        
        self.pdf_button = ttk.Button(pdf_frame, text="Browse...", command=self.browse_pdf)
        self.pdf_button.pack(side=tk.LEFT, padx=5)
        
        # Summary button
        self.pdf_summarize_button = ttk.Button(self.pdf_tab, text="Summarize", 
                                              command=lambda: self.summarize("pdf"))
        self.pdf_summarize_button.pack(pady=10)
        self.pdf_summarize_button.config(state=tk.DISABLED)  # Disabled until PDF is loaded
        
        # Output area
        output_frame = ttk.LabelFrame(self.pdf_tab, text="Summary")
        output_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        self.pdf_output = scrolledtext.ScrolledText(output_frame, height=10, wrap=tk.WORD)
        self.pdf_output.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    def browse_pdf(self):
        file_path = filedialog.askopenfilename(
            title="Select a PDF file",
            filetypes=[("PDF files", "*.pdf")]
        )
        
        if not file_path:
            return
        
        filename = os.path.basename(file_path)
        self.pdf_label.config(text=f"Selected: {filename}")
        self.status_var.set(f"Loading PDF: {filename}")
        self.root.update()
        
        try:
            self.pdf_text = self.extract_text_from_pdf(file_path)
            word_count = len(self.pdf_text.split())
            self.status_var.set(f"PDF loaded: {word_count} words extracted")
            
            self.pdf_summarize_button.config(state=tk.NORMAL)
            
            if word_count > 1000:
                if messagebox.askyesno("Long Text Warning", 
                                      f"The PDF contains {word_count} words which is quite long. Do you want to use only the first 1000 words?"):
                    self.pdf_text = ' '.join(self.pdf_text.split()[:1000])
                    self.status_var.set(f"Using first 1000 words of PDF")
        except Exception as e:
            self.status_var.set(f"Error processing PDF: {str(e)}")
            messagebox.showerror("PDF Error", f"Could not process PDF: {str(e)}")
            self.pdf_label.config(text="No PDF selected")
            self.pdf_summarize_button.config(state=tk.DISABLED)
    
    def extract_text_from_pdf(self, file_path):
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                
                # Extract text from each page
                for page in pdf_reader.pages:
                    text += page.extract_text() + " "
                    
                return text
        except Exception as e:
            raise Exception(f"Error extracting text: {str(e)}")
    
    def summarize(self, input_type):
        if input_type == "text":
            text = self.text_input.get(1.0, tk.END).strip()
            output_widget = self.text_output
        else:  # pdf
            text = self.pdf_text
            output_widget = self.pdf_output
        
        # Clear previous output
        output_widget.delete(1.0, tk.END)
        
        if not text:
            messagebox.showwarning("No Input", "Please enter text or select a PDF file.")
            return
        
        self.status_var.set("Generating summary...")
        self.root.update()
        
        try:
            summary = self.summarise_text(text)
            output_widget.insert(tk.END, summary)
            self.status_var.set("Summary generated successfully")
        except Exception as e:
            error_msg = str(e)
            output_widget.insert(tk.END, f"Error generating summary: {error_msg}")
            self.status_var.set("Error generating summary")
    
    def summarise_text(self, text):
        try:
            model_name = "models/gemini-1.5-flash"
            
            model = genai.GenerativeModel(model_name)
            
            prompt = f"""Summarise the following text in simple and easy language.
            
Your summary should:
- Be about the same length as a single sentence or two short sentences
- Simplify technical terms but retain the key concepts
- Include the main purpose or function described
- Be understandable by a middle-school student

Now summarize this text:
{text}"""
            
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            # If the first attempt fails, try with a different model
            try:
                backup_model_name = "models/gemini-2.0-flash"
                
                backup_model = genai.GenerativeModel(backup_model_name)
                response = backup_model.generate_content(prompt)
                return response.text.strip()
            except Exception as e2:
                raise Exception(f"Primary error: {str(e)}\nBackup error: {str(e2)}")

def main():
    root = tk.Tk()
    app = SummarizerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()